In [15]:
import subprocess
import altair as alt
import pandas as pd

In [16]:
# In order to run clingo command in the terminal and capture the output
def run_clingo(puzzle):
    result = subprocess.run(['clingo', puzzle, 'program.lp', '--stats'], 
                            capture_output=True, text=True)
    output = result.stdout
    return output

In [17]:
def analyze_output(output):
    lines = output.split('\n')
    stats = {}
    for line in lines:
        if "CPU Time" in line:
            stats["time"] = float((line.split(":")[1].strip()).split("s")[0])
        elif "Rules" in line:
            stats["rules"] = int((line.split(":")[1].strip()).split(" ")[0])
        elif "Atoms" in line:
            stats["atoms"] = int((line.split(":")[1].strip()).split(" ")[0])

    return stats

In [18]:
def classify_puzzle(stats):
    if stats["time"] < 0.2 and stats["atoms"] < 3000:
        return "Very easy"
    elif stats["time"] < 0.5:
        return "Easy"
    elif stats["time"] < 1:
        return "Medium"
    elif stats["time"] < 2:
        return "Hard"
    else:
        return "Harder"

In [19]:
def analyze_puzzles(puzzles):
    results = []
    i = 1
    for puzzle in puzzles:
        output = run_clingo(puzzle)
        stats = analyze_output(output)
        difficulty = classify_puzzle(stats)
        stats["difficulty"] = difficulty
        stats["puzzle"] = puzzle
        stats["size"] = i
        i += 1
        results.append(stats)

    return results

In [20]:
puzzles = ["param_gen1.db", "param_gen2.db","param_gen3.db", "param_gen4.db", "param_gen5.db",
            "param_gen6.db", "param_gen7.db", "param_gen8.db",
              "param_gen9.db"]
puzzles_results = analyze_puzzles(puzzles)
df = pd.DataFrame(puzzles_results)
print(df)

    time    rules  atoms difficulty         puzzle  size
0  0.005     1027    623  Very easy  param_gen1.db     1
1  0.014     8925   1471  Very easy  param_gen2.db     2
2  0.044    33444   2508  Very easy  param_gen3.db     3
3  0.130    83869   4175       Easy  param_gen4.db     4
4  0.264   175770   5776       Easy  param_gen5.db     5
5  0.508   311504   8037     Medium  param_gen6.db     6
6  0.865   502678  10676     Medium  param_gen7.db     7
7  1.344   752680  13963       Hard  param_gen8.db     8
8  2.006  1090672  17130     Harder  param_gen9.db     9


In [21]:
chart = alt.Chart(df).mark_line().encode(
    alt.X('size', type='ordinal', title='Puzzle size'),
    alt.Y('time', type='quantitative', title='Time taken (s)'),
).properties(title='Time taken to solve puzzles').interactive()
chart.show()

alt.Chart(...)